In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os

import anndata as ad
import numpy as np
import pandas as pd
import sklearn
import torch
import tqdm

import celltrip

os.environ['AWS_PROFILE'] = 'waisman-admin'


# Load Data and Policy

In [3]:
# Read data files
# adata_prefix = 's3://nkalafut-celltrip/Dyngen'
adata_prefix = '../data/Dyngen'
adatas = celltrip.utility.processing.read_adatas(
    's3://nkalafut-celltrip/dyngen/logcounts.h5ad',
    's3://nkalafut-celltrip/dyngen/counts_protein.h5ad',
    backed=True)
# Model location and name (should be prefix for .weights, .pre, and .mask file)
prefix, training_step = 's3://nkalafut-celltrip/checkpoints/Dyngen-250920', 800
# Generate or load preprocessing
preprocessing = celltrip.utility.processing.Preprocessing().load(f'{prefix}.pre')
with celltrip.utility.general.open_s3_or_local(f'{prefix}.mask', 'rb') as f:
    mask = np.loadtxt(f).astype(bool)
adatas[0].obs['Training'] = mask
# Create sample env (kind of a dumb workaround, TODO)
m1, m2 = [preprocessing.transform(ad[:2].X, subset_modality=i)[0] for i, ad in enumerate(adatas)]
env = celltrip.environment.EnvironmentBase(
    torch.tensor(m1), torch.tensor(m2), compute_rewards=False, dim=8).eval().to('cuda')
# Load policy
policy = celltrip.policy.create_agent_from_env(
    env, forward_batch_size=1_000, vision_size=1_000).eval().to('cuda')
policy.load_checkpoint(f'{prefix}-{training_step:04}.weights');


# Perform Knockdown

In [ ]:
# Params
np.random.seed(42)
genes_to_survey = adatas[0].var_names
sim_time = 5.

# Mute warnings (array wrap and indexing)
import warnings
warnings.simplefilter('ignore')

# Create anndata
ad_pert = ad.AnnData(obs=adatas[0].obs, var=pd.DataFrame(index=[f'Feature {i}' for i in range(env.dim)]))
# ad_pert0 = ad.AnnData(obs=adatas[0].obs, var=adatas[0].var)
# ad_pert1 = ad.AnnData(obs=adatas[1].obs, var=adatas[1].var)
def add_layers(states, gene):
    ad_pert.layers[gene] = states
    # ad_pert0.layers[gene] = states_0
    # ad_pert1.layers[gene] = states_1

# Add results
results = []
def add_record(states, states_0, states_1, gene, ct):
    results.append({
        'Gene': gene, 'Cell Type': ct,
        'Effect Size (Latent)': np.square(states[-1] - states[0]).mean(),
        'Trajectory Length (Latent)': np.square(states[1:] - states[:-1]).mean(axis=(-2, -1)).sum(),
        'Effect Size (Modality 0)': np.square(states_0[-1] - states_0[0]).mean(),
        'Trajectory Length (Modality 0)': np.square(states_0[1:] - states_0[:-1]).mean(axis=(-2, -1)).sum(),
        'Effect Size (Modality 1)': np.square(states_1[-1] - states_1[0]).mean(),
        'Trajectory Length (Modality 1)': np.square(states_1[1:] - states_1[:-1]).mean(axis=(-2, -1)).sum()})
    
# Reset function
def reset_env(env, steady_pos, steady_vel, modal_dict={}):
    env.set_max_time(sim_time).reset()  # TODO: Maybe longer?, early stopping?
    env.set_positions(steady_pos)
    env.set_velocities(steady_vel)  # Maybe 0 manually?
    for k, v in modal_dict.items():
        env.modalities[k] = v

# Running function
def run_and_record(samples, env, policy, preprocessing, gene):
    # Run and impute
    states = celltrip.train.simulate_until_completion(env, policy, store_states='cpu')[-1]
    states_pos = states[..., :env.dim]
    with torch.no_grad():
        imputed_states_0 = policy.pinning[0](states_pos.to('cuda')).detach().cpu().numpy()
        imputed_states_1 = policy.pinning[1](states_pos.to('cuda')).detach().cpu().numpy()
    imputed_states_0, = preprocessing.inverse_transform(imputed_states_0, subset_modality=0)
    imputed_states_1, = preprocessing.inverse_transform(imputed_states_1, subset_modality=1)
    # Record
    add_layers(states_pos.numpy()[-1], gene)  # , imputed_states_0[-1], imputed_states_1[-1]
    add_record(states_pos.numpy(), imputed_states_0, imputed_states_1, gene, 'All')
    for ct in adatas[0][samples].obs['traj_sim'].unique():
        add_record(
            states_pos[:, adatas[0][samples].obs['traj_sim']==ct].numpy(),
            imputed_states_0[:, adatas[0][samples].obs['traj_sim']==ct],
            imputed_states_1[:, adatas[0][samples].obs['traj_sim']==ct],
            gene, ct)

# Subset and preprocess the data
samples = adatas[0].obs.index
raw_m1 = celltrip.utility.processing.chunk_X(adatas[0][samples], chunk_size=2_000)
m1, m2 = [
    celltrip.utility.processing.chunk_X(
        ad[samples], chunk_size=2_000,
        func=lambda x: preprocessing.transform(x, subset_modality=i)[0])
        for i, ad in enumerate(adatas)]

# Initialize environment
env = celltrip.environment.EnvironmentBase(
    torch.tensor(m1), torch.tensor(m2), compute_rewards=False, dim=8).eval(time_scale=1).to('cuda')

# Simulate to steady state
env.reset()
celltrip.train.simulate_until_completion(env, policy)
steady_pos, steady_vel = (env.pos, env.vel)

# Run control
reset_env(env, steady_pos, steady_vel)
add_layers(steady_pos.cpu().numpy(), 'Steady')
run_and_record(samples, env, policy, preprocessing, 'Control')

# Perturb
for gene in tqdm.tqdm(genes_to_survey, miniters=10, maxinterval=30):
    # Get gene idx and reset environment
    gene_idx = np.argwhere(adatas[0].var_names==gene).flatten()
    reset_env(env, steady_pos, steady_vel)  # {0: torch.tensor(m1).cuda()}

    # Apply knockdown
    # iso_modality, = preprocessing.transform(raw_m1, subset_features=gene_idx, subset_modality=0)
    iso_modality = celltrip.utility.processing.chunk(
        raw_m1, chunk_size=2_000, func=lambda x: preprocessing.transform(x, subset_features=gene_idx, subset_modality=0)[0])
    iso_modality = torch.tensor(iso_modality).to(env.device)
    modal_change = (iso_modality - 0*iso_modality)  # Coeff is for overexp anal
    env.modalities[0] = env.modalities[0] - modal_change

    # Simulate and record
    run_and_record(samples, env, policy, preprocessing, gene)
    env.modalities[0] = env.modalities[0] + modal_change

# Convert and save
pd.DataFrame(results).to_csv('../plots/dyngen/knockdown.csv', index=None)
ad_pert.write_h5ad('../plots/dyngen/knockdown_results.h5ad')
# ad_pert0.write_h5ad('../plots/dyngen/knockdown_results_modality_0.csv')
# ad_pert1.write_h5ad('../plots/dyngen/knockdown_results_modality_1.csv')



  0%|                                                                                                                                                                                              | 0/2400 [00:00<?, ?it/s]


  0%|▋                                                                                                                                                                                  | 10/2400 [00:29<1:56:29,  2.92s/it]


  1%|█▍                                                                                                                                                                                 | 20/2400 [00:58<1:55:42,  2.92s/it]


  1%|██▏                                                                                                                                                                                | 30/2400 [01:27<1:55:21,  2.92s/it]


  2%|██▉                                                                                                                                                                                | 40/2400 [01:56<1:54:44,  2.92s/it]

...


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 2370/2400 [1:54:24<01:27,  2.91s/it]


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 2380/2400 [1:54:53<00:58,  2.91s/it]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 2390/2400 [1:55:22<00:29,  2.90s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2400/2400 [1:55:51<00:00,  2.91s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2400/2400 [1:55:51<00:00,  2.90s/it]

In [5]:
# knockdown = pd.read_csv('../plots/dyngen/knockdown.csv', index_col=None)
# # knockdown.groupby(['Gene', 'Cell Type']).first().head(30)
# ad_pert = ad.read_h5ad('../plots/dyngen/knockdown_results.h5ad')
# ad_pert
